<p align="center">
    <img width="20%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Il_Post_logo.svg/800px-Il_Post_logo.svg.png">
</p>
Dataset per Abstract Summarization

In [ ]:
import requests

# pagina
r = requests.get("https://www.ilpost.it/economia")

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, "html5lib")
articles = soup.find_all("article")

print(BeautifulSoup(str(articles[6]), 'html.parser').prettify())

In [ ]:
summary = []
for i in range(len(articles)):
    try: 
      testo1 = articles[i].select_one("h1 a, h2 a, h3 a").get_text().strip()
    except:
      ""
    try:
      testo2 = articles[i].select_one("p").get_text().strip()
      testo2 = testo2[0].lower() + testo2[1:]
    except:
      ""
    riassunto = testo1 + " " + testo2
    print(str(i) + ": " + riassunto)
    summary.append((riassunto))

In [ ]:
pagelinks = []
for link in articles:
  url = link.find_all('a')[0]
  pagelinks.append(url.get('href'))

In [ ]:
print(len(articles), len(pagelinks))

In [ ]:
thearticle = []
paragraph_test = []
for link in pagelinks:    
    paragraphtext = []    
    url = link
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')        

    try:
      testo_articolo = soup.find_all("p")[1:4]
    except:
      testo_articolo = ""
    for paragraph in testo_articolo[:-1]:
        # get the text only
        text = paragraph.get_text()
        paragraphtext.append(text)

    testo_fin = ' '.join(paragraphtext)     
    thearticle.append(testo_fin)


In [ ]:
print("Titolo: \n" + summary[4], "\n\nArticolo: \n" + thearticle[4] + "\n")

# Dataset

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

Num. of pages for every topic (from website)



*   Tecnologia: 97
*   Politica: 133
*   Italia: 414
*   Cultura: 593
*   Economia: 148
*   Scienza: 148
*   Sport: 381


In [ ]:
argo = ["economia", "tecnologia", "politica", "italia", "cultura", "scienza", "sport", "internet"]
popol = [148, 97, 184, 414, 594, 149, 381, 114]

In [ ]:
#@title ## Topic
#@markdown Choose.

#@markdown ---
argomento = "italia" #@param ["economia", "tecnologia", "politica","italia","cultura","scienza","sport","internet"]
argomenti = [argomento]
pop = [popol[argo.index(argomento)]]

In [ ]:
import time

def coppie(url):
  time.sleep(1)
  r = requests.get(url)
  soup = BeautifulSoup(r.text, "html5lib")
  articles = soup.find_all("article")
  
  # list of summaries
  summary = []
  for i in range(len(articles)):
    try: 
      testo1 = articles[i].select_one("h1 a, h2 a, h3 a").get_text().strip()
    except:
      ""
    try:
      testo2 = articles[i].select_one("p").get_text().strip()
    except:
      ""

    if testo1[-1] == "?" or testo1[-1] == ".": 
      if testo2 == "": riassunto = testo1
      elif testo2[-1] == ".": riassunto = testo1 + " " + testo2
      else: riassunto = testo1 + " " + testo2[0].lower() + testo2[1:] + "."
    elif testo1[-1] == ",":
      if testo2 == "": riassunto = testo1[:-1] + "."
      elif testo2[-1] == ".": riassunto = testo1 + " " + testo2[0].lower() + testo2[1:]
      else: riassunto = testo1 + " " + testo2[0].lower() + testo2[1:] + "."
    elif testo1[-1] != ",": 
      if testo2 == "": riassunto = testo1 + "."
      elif testo2[-1] == ".": riassunto = testo1 + ". " + testo2
      else: riassunto = testo1 + ". " + testo2 + "."
    summary.append((riassunto))

  # articles urls
  pagelinks = []
  for link in articles:
    url = link.find_all('a')[0]
    pagelinks.append(url.get('href'))

  thearticle = []
  paragraph_test = []
  for link in pagelinks:    
    paragraphtext = []    
    url = link
    page = requests.get(url)
    time.sleep(0.1)
    soup = BeautifulSoup(page.text, 'html.parser')        

    try:
      testo_articolo = soup.find_all("p")[1:4]
    except:
      testo_articolo = ""
    for paragraph in testo_articolo[:-1]:
        text = paragraph.get_text()
        paragraphtext.append(text)

    testo_fin = ' '.join(paragraphtext)     
    thearticle.append(testo_fin)
  d = {'source': thearticle, 'target': summary}
  return d

In [ ]:
# get topic summaries and articles
df = pd.DataFrame(columns=["source", "target"])
i = 0
j = 0
conto = 0
for i in range(len(argomenti)):
  for j in range(1, pop[i]):
    if j == 1:      
      ret = coppie("https://www.ilpost.it/" + argomenti[i]+"/")
      conto += 26
      print("\nPagina: " + "https://www.ilpost.it/" + argomenti[i]+"/"
      + "\nArticoli Scaricati: " + str(conto))
      df = pd.concat([df, pd.DataFrame(ret)])
    else: 
      ret = coppie("https://www.ilpost.it/" + argomenti[i] + "/page/" + str(j) +"/")
      conto += 26
      print("\nPagina: " + "https://www.ilpost.it/" + argomenti[i] + "/page/" + str(j) +"/"
      + "\nArticoli Scaricati: " + str(conto))
      #ret = coppie("https://www.ilpost.it/%s/page/%s/"%(argomenti[i], str(j)))
      df = pd.concat([df, pd.DataFrame(ret)])

In [ ]:
df.shape[0]

In [ ]:
import numpy as np

# some cleaning, remove empty rows 
df['source'].replace("", np.nan, inplace=True)
df['target'].replace("", np.nan, inplace=True)
df.dropna(subset=['source'], inplace=True)
df.dropna(subset=['target'], inplace=True)
df = df.reset_index(drop=True)

print(df.source.size, df.target.size)

In [ ]:
df.to_csv("/content/IlPost_Italia.csv", sep=',', index=False)

### plots

In [ ]:
import re
word_bound = re.compile(r'\b')

def num_words(line):
    return len(word_bound.findall(line)) >> 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') 
from matplotlib.colors import to_rgba

In [ ]:
ii = [i for i in range(0,80)]
lunghezze_riass = [num_words(df["target"].iloc[i]) for i in range(df.shape[0])]

fig = plt.figure()
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

fig.suptitle('lunghezze riassunti', fontsize=14, fontweight='bold')
ax.set_title('IlPost')

ax.set_xlabel('tokens')
ax.set_ylabel('numero')

#plt.style.use(['dark_background'])
plt.hist(lunghezze_riass, bins=ii, color='c')

plt.show()

In [ ]:
ii = [i for i in range(0,500,6)]
lunghezze_docu = [num_words(df['source'].iloc[i]) for i in range(df.shape[0])]

fig = plt.figure()
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

fig.suptitle('lunghezze documenti', fontsize=14, fontweight='bold')
ax.set_title('IlPost')

ax.set_xlabel('tokens')
ax.set_ylabel('numero')

#plt.style.use(['dark_background'])
plt.hist(lunghezze_docu, bins=ii, color='c')
plt.show()

# Text cleaning

We used textacy for text cleaning, normalizing unicode, removing html tags and normalizing whitespaces 

In [ ]:
!pip install -q textacy 

In [ ]:
import pandas as pd

# read dataset csv 
df = pd.read_csv("IlPost_Tech.csv")
df.shape[0]

In [ ]:
from textacy import preprocessing
clean_inputs = []
clean_targets = []

preproc = preprocessing.make_pipeline(
     preprocessing.normalize.unicode,
     preprocessing.remove.html_tags,
     preprocessing.normalize.whitespace
)

# apply to dataset elements
clean_inputs = [preproc(df["source"].iloc[i]) for i in range(df.shape[0])]
clean_targets = [preproc(df["target"].iloc[i]) for i in range(df.shape[0])]
df = pd.DataFrame({"source": clean_inputs, "target": clean_targets})

In [ ]:
df.to_csv("/content/IlPost_Italia.csv", sep=',', index=False)